In [7]:
import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM
from urllib.request import urlopen
import torch.nn as nn
from huggingface_hub import hf_hub_download

import matplotlib.pyplot as plt

# Loading some sources of the projection adapter and image encoder
#hf_hub_download(repo_id="AIRI-Institute/OmniFusion", filename="models.py", local_dir='./')
#from models import CLIPVisionTower

DEVICE = "cuda:1"
PROMPT = "This is a dialog with AI assistant.\n"

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",torch_dtype=torch.bfloat16, device_map=DEVICE)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

unk_id = tokenizer.encode("<unk>", add_special_tokens=False)[0]
tokenizer.pad_token_id = 2
tokenizer.eos_token_id = 0

model.resize_token_embeddings(len(tokenizer))
N_EMBEDDINGS = model.model.embed_tokens.weight.shape[0]
print("Number of embeddings in tokenizer:", N_EMBEDDINGS)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Number of embeddings in tokenizer: 32001


In [1]:
from datasets import load_dataset

ds = load_dataset("Sayankotor/small_wikipaper")

Using the latest cached version of the dataset since Sayankotor/small_wikipaper couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/jovyan/.cache/huggingface/datasets/Sayankotor___small_wikipaper/default/0.0.0/5ee56d00d12a264ade040d1e7ed370770856b3fe (last modified on Wed Mar 13 01:59:29 2024).


Loading dataset shards:   0%|          | 0/53 [00:00<?, ?it/s]

In [12]:
ds['train'][67]

{'text': 'In law, an abstract is a brief statement that contains the most important points of a long legal document or of several related legal papers. ==Types of legislation == The abstract of title, used in real estate transactions, is the more common form of abstract. An abstract of title lists all the owners of a piece of land, a house, or a building before it came into possession of the present owner. The abstract also records all deeds, wills, mortgages, and other documents that affect ownership of the property. An abstract describes a chain of transfers from owner to owner and any agreements by former owners that are binding on later owners. ==Patent law== In the context of patent law and specifically in prior art searches, searching through abstracts is a common way to find relevant prior art document to question to novelty or inventive step (or non-obviousness in United States patent law) of an invention. Under United States patent law, the abstract may be called "Abstract of 

In [6]:
a

,text,text_with_Q,entities,entity_numbs,entity_embs
0,Alabama () is a state in the Southeastern regi...,Alabama () is a state in the Southeastern regi...,"['Dixie', 'Huntsville, Alabama', 'Mount Cheaha...","['Q721983', 'Q79860', 'Q927744', 'Q12630', 'Q1...","[[-0.6626, 0.3285, 0.5655, 0.0, -0.3768, 0.013..."
1,"Abraham Lincoln ( ; February 12, 1809 – April ...","#QAbraham Lincoln ( ; February 12, 1809 – Apri...","['Hannibal Hamlin', 'James Buchanan', 'Andrew ...","['Q273546', 'Q12325', 'Q8612', 'Q1204', 'Q9089...","[[0.1798, 0.2912, 0.1084, -0.1131, -0.1883, 0...."
2,"Aristotle (; Aristotélēs, ; 384–322 BC) was an...","Aristotle (; Aristotélēs, ; 384–322 BC) was an...","['Lysippos', 'Chalcis', 'Euboea', 'Peripatetic...","['Q192222', 'Q200133', 'Q173096', 'Q194414', '...","[[-0.2045, 0.3586, 0.6805, -0.0497, 0.5481, 0...."
3,An American in Paris is a jazz-influenced symp...,An American in #QParis is a jazz-influenced sy...,"['George Gershwin', 'Carnegie Hall', 'Walter D...","['Q123829', 'Q200959', 'Q725579', 'Q471154', '...","[[0.2848, 0.3715, 0.1362, -0.0089, -0.2748, 1...."
4,The Academy Award for Best Production Design r...,The Academy Award for Best Production Design r...,['Academy of Motion Picture Arts and Sciences'...,"['Q212329', 'Q248688', 'Q727904', 'Q2162645', ...","[[-0.1671, 0.0591, -0.0517, 0.3575, -0.3635, -..."
...,...,...,...,...,...
327688,The Aberdeen IronBirds are a Minor League Base...,The Aberdeen IronBirds are a #QMinor League Ba...,"['High-A', 'South Atlantic League', 'New York–...","['Q22994009', 'Q925256', 'Q1135467', 'Q650816'...","[[-0.2768, 0.5025, 0.17, 0.5021, 0.4888, -0.09..."
327689,Catch as catch can may refer to: *Catch wrestl...,Catch as catch can may refer to: *#QCatch wres...,"['Catch wrestling', 'Catch as Catch Can (album...","['Q2408025', 'Q2941730', 'Q5051691', 'Q5051690...","[[-0.4093, 0.6189, -0.1183, 0.1758, -0.2641, -..."
327690,Interior architecture is the design of a build...,Interior architecture is the design of a build...,"['Sustainable design', 'Rhode Island School of...","['Q523256', 'Q49208', 'Q3445331', 'Q432637', '...","[[-0.12, 0.3311, 0.1976, -0.0017, -0.3708, -0...."
327691,is a Japanese manga series written by Akira It...,is a Japanese manga series written by Akira It...,"['Shueisha', 'Viz Media', 'V Jump', 'Kazuki Ta...","['Q844822', 'Q660288', 'Q81073', 'Q240225', 'Q...","[[0.0015, -0.0786, 0.0468, 1.0724, 0.2042, 0.0..."


In [9]:
print (a.iloc[67])

text            In law, an abstract is a brief statement that ...
text_with_Q     In law, an abstract is a brief statement that ...
entities        ['United States patent law', 'United States Pa...
entity_numbs                 ['Q3039740', 'Q1459541', 'Q1056771']
entity_embs     [[-0.0427, -0.0064, 0.2264, -0.5276, -0.4055, ...
Name: 67, dtype: object


In [29]:
import numpy as np
np.mean(lens)

39.326540389938145

In [28]:
import numpy as np
print (np.max(lens), np.min(lens))

6275 0


In [26]:
from ast import literal_eval
len(np.array(literal_eval(ds['train'][10]['entities'])))

52

In [ ]:
from ast import literal_eval
lens = []
for ind, elem in enumerate(ds['train']):
    if (ind % 1000 == 0):
        print (ind)
    try:
        lens.append(len(np.array(literal_eval(elem['entities']))))
        if len(np.array(literal_eval(elem['entities']))) < 8:
            print (ind)
    except:
        print(elem)

In [17]:
ds['train'][10]['entities']

'[\'Draguignan\', \'Collège de France\', \'Ohio State University\', \'Vanderbilt University\', \'École Normale Supérieure\', \'Pierre and Marie Curie University\', \'Jacques Dixmier\', \'Baum–Connes conjecture\', \'Fields Medal\', \'Clay Research Award\', \'Crafoord Prize\', \'Marseille\', \'Paris\', \'French National Centre for Scientific Research\', "Queen\'s University at Kingston", \'Conservatoire national des arts et métiers\', \'Connes embedding problem\', \'K-theory\', \'Nicolas Bourbaki\', \'American Mathematical Society\', \'International Congress of Mathematicians\', \'University of Rome Tor Vergata\', \'University of Oslo\', \'University of Southern Denmark\', \'Fudan University\', \'French Academy of Sciences\', \'Royal Danish Academy of Sciences and Letters\', \'Norwegian Academy of Science and Letters\', \'American Academy of Arts and Sciences\', \'London Mathematical Society\', \'Royal Society of Canada\', \'National Academy of Sciences\', \'Russian Academy of Sciences\'

In [10]:
len(ds['train'])

327693

In [11]:
from torch.utils.data import DataLoader, Dataset
from ast import literal_eval
import numpy as np


class PretrainDataset(Dataset):
    def __init__(self, ds):
        self.ds = ds['train']

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        try:
            text = self.ds[idx]['text'][:2048]
        except:
            text = "Bad example"
            print (self.ds[idx])
        ents = self.ds[idx]['entities']
        embs = np.array(literal_eval(self.ds[idx]['entity_embs']))[...,:200]
        #image = Image.open(path).convert("RGB")
        return text, ents, embs

In [12]:
dataset = PretrainDataset(ds)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [13]:
a, b, c = dataset[5]

In [14]:
type(ds['train'][5]['entity_embs'])

str

In [15]:
list_of_embs = literal_eval(ds['train'][5]['entity_embs'])

In [16]:
len(list_of_embs)

17

In [17]:
import numpy as np
np.array(c).shape

(17, 200)

In [18]:
len(dataloader)

327693

### Train adapter

In [19]:
tokenizer.vocab_size

32000

In [20]:
1205/2

602.5

In [21]:
batch = next(iter(dataloader))

In [22]:
a = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))

In [23]:
a.shape

torch.Size([2, 3])

In [24]:
a[:,:-1].shape

torch.Size([2, 2])

In [25]:
a, b, c = batch

In [26]:
bad_words_ids = tokenizer(["\n", "</s>", ":"], add_special_tokens=False).input_ids + [[13]]
gen_params = {
        "do_sample": False,
        "max_new_tokens": 40,
        "early_stopping": False,
        "num_beams": 3,
        "repetition_penalty": 1.0,
        "remove_invalid_values": True,
        "eos_token_id": 2,
        "pad_token_id": 2,
        "forced_eos_token_id": 2,
        "use_cache": True,
        "no_repeat_ngram_size": 4,
        "bad_words_ids": bad_words_ids,
        "num_return_sequences": 1,
    }

In [ ]:
import torch
from torch import nn
from torch.optim import AdamW
from transformers.optimization import (Adafactor, AdafactorSchedule,
                                       get_cosine_schedule_with_warmup)

import gc

import tqdm

kg_emb_dim = 200
mstral_emb_dim = 4096

kg_start_emb = torch.normal(torch.zeros(mstral_emb_dim), torch.ones(mstral_emb_dim) / mstral_emb_dim**0.5).to(device=DEVICE, dtype=model.dtype)
kg_end_emb = torch.normal(torch.zeros(mstral_emb_dim), torch.ones(mstral_emb_dim) / mstral_emb_dim**0.5).to(device=DEVICE, dtype=model.dtype)
projection = nn.Linear(kg_emb_dim, mstral_emb_dim).to(device=DEVICE, dtype=model.dtype)

kg_start_emb.requires_grad_()
kg_end_emb.requires_grad_()
model.requires_grad_(False)

    
lr = 5e-3
weight_decay = 1e-3
trainable_parameters = [kg_start_emb] + [kg_end_emb] + list(projection.parameters())

opt = AdamW(trainable_parameters, lr=lr, weight_decay=weight_decay)
loss_fct = nn.CrossEntropyLoss(reduction="none", ignore_index=unk_id)

grad_accum = 256

loss_best = 1000.0

losses = []
losses_batch = []
iters = 0
n_iters = len(dataloader)
scheduler = get_cosine_schedule_with_warmup(opt, num_warmup_steps=n_iters // grad_accum * 0.01, num_training_steps=n_iters // grad_accum)

for epoch in range(1):
    i = 0 
    for step in tqdm.notebook.tqdm(range(n_iters)):
        batch = next(iter(dataloader))
        text, ents, embs = batch
        model.eval()
        model.requires_grad = False
        opt.zero_grad()
        with torch.no_grad():
            text_ids = tokenizer.encode(text[0], add_special_tokens=False, return_tensors="pt")[..., :2048].to(device=DEVICE)
            
            input_embeddings = model.model.embed_tokens(text_ids[...,:text_ids.shape[1]//2])
            output_embeddings = model.model.embed_tokens(text_ids[...,text_ids.shape[1]//2:])
            
        try:
            m = batch[2].mean(2, keepdim=True)
            s = batch[2].std(2, unbiased=False, keepdim=True)
            batch[2] -= m
            batch[2] /= s
        except:
            print (batch[2].shape)
        try:
            projected_kg_embeddings = projection(batch[2].to(
                        device=DEVICE, dtype=model.dtype
                    ))
        except:
            print ("embs.shape", batch[2].shape)
            continue
        
        embeddings1 = torch.cat(
                [
                    input_embeddings,
                    kg_start_emb[None, None, ...],
                    projected_kg_embeddings,
                    kg_end_emb[None, None, ...],
                    output_embeddings
                ],
                dim=1,
            )
        
        gen_params['max_new_tokens'] = embeddings1.shape[1]
        
        #mask = torch.full(embeddings1.shape, False)
        
        with torch.autocast(device_type="cuda", dtype=model.dtype):
            logits = model(inputs_embeds=torch.cat(
                [
                    input_embeddings,
                    kg_start_emb[None, None, ...],
                    projected_kg_embeddings,
                    kg_end_emb[None, None, ...],
                    output_embeddings
                ],
                dim=1,
            ), output_hidden_states=True).get("logits")
            # loss only for answer part & backward
            #print ("logits.shape", logits.shape)
            #print ("output_embeddings", output_embeddings.shape)
            logits = logits[..., -output_embeddings.shape[1]:-1, :].contiguous()
            labels = text_ids[...,text_ids.shape[1]//2 + 1:].contiguous()
            #labels = labels[...,:text_ids.shape[1]//2]
            
            #mask = mask[:, -output_embeddings.shape[1]:]
        
            loss = loss_fct(logits.permute(0, 2, 1), labels).mean()
            #print (loss)
            
        if model.dtype == torch.float16:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        losses_batch.append(loss.item())
        
        if (step% 1000 == 0):         
            out = model.generate(inputs_embeds=embeddings1, **gen_params)
            #print ("out.shape", out.shape)
            #print ("projected_kg_embeddings shape", projected_kg_embeddings.shape)
            #print ("out.shape", out.shape)
            generated_texts1 = tokenizer.batch_decode(out[:, 1:])[0]
            generated_texts2 = tokenizer.batch_decode(out)[0]
            print ("\n first part \n")
            print (tokenizer.decode(token_ids=text_ids[...,:text_ids.shape[1]//2][0]))
            print ("\n last part \n")
            print (tokenizer.decode(token_ids=text_ids[...,text_ids.shape[1]//2:][0]))
            print ("\n continue", generated_texts1)
            print ("\n")
            print ("\n continue", generated_texts2)
            print ("\n")
            
            print ("loss", np.mean(losses_batch))
            print ('lr', scheduler.get_lr()[0], step, flush = True)
            plt.title("train loss\n" + f"\n\nEpoch [{epoch}], iter [{iters}/{n_iters}]")
            accum_loss = np.mean(losses_batch)
            losses.append(accum_loss)
            plt.semilogy(losses)
            plt.grid()
            plt.savefig(f"ckpts/loss1.png")
            plt.close("all")


        if iters % grad_accum == 0 and iters > 0:
            if model.dtype == torch.float16:
                scaler.step(opt)
                scaler.update()
            else:
                opt.step()
            opt.zero_grad()
            scheduler.step()
            accum_loss = np.mean(losses_batch)
            losses.append(accum_loss)
            losses_batch = []

            if accum_loss < loss_best:
                loss_best = accum_loss
                torch.save(projection, f"ckpts/projection_llama2_chat")
                torch.save(kg_start_emb, f"ckpts/SOI_llama2_chat.pt")
                torch.save(kg_end_emb, f"ckpts/EOI_llama2_chat.pt")
            
            
            #gc.collect()
        
        iters += 1

        # model inference to get
        
        

  0%|          | 0/327693 [00:00<?, ?it/s]

/home/jovyan/.mlspace/envs/Python310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jovyan/.mlspace/envs/Python310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



 first part 

In the Skin of a Lion is a novel by Canadian–Sri Lankan writer Michael Ondaatje. It was first published in 1987 by McClelland and Stewart. The novel fictionalizes the lives of the immigrants who played a large role in the building of the city of Toronto in the early 1900s, but whose contributions never became part of the city's official history.Devi, S. Poorna Mala. "Immigrants' experience in Michael Ondaatje's novels in the skin of a lion and the English patient." Language In India, January 2015, 547+. Literature Resource Center (accessed December 1, 2016). http://go.galegroup.com/ps/i.do?p=GLS&sw;=w&u;=ocul_carleton&v;=2.1⁢=r&id;=GALE%7CA404830601&asid;=61174144a6b42fbc8556f9c27c32c1c3 Ondaatje illuminates the investment of these settlers in Canada, through their labour, while they remain outsiders to mainstream society. In the Skin of a

 last part 

Lion is thus an exposé of the migrant condition: "It is a novel about the wearing and the removal of masks; the sheddin

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



torch.Size([1, 0])
embs.shape torch.Size([1, 0])
torch.Size([1, 0])
embs.shape torch.Size([1, 0])

 first part 

Joseph Jules François Félix Babinski (; 17 November 1857 – 29 October 1932) was a French-Polish professor of neurology. He is best known for his 1896 description of the Babinski sign, a pathological plantar reflex indicative of corticospinal tract damage. ==Life== Born in Paris, Babinski was the son of a Polish military officer, Aleksander Babiński (1824–1889), and his wife Henryeta Weren Babińska (1819–1897),Joseph Babinski. nndb.com who in 1848 fled Warsaw for Paris because of a Tsarist reign of terror instigated to stall Polish attempts at achieving independence and breaking the union between Congress Poland and the Russian Empire. Babinski received his medical degree from the University of Paris in 1884. He came early to Professor Charcot at Paris' Salpêtrière Hospital and became his favorite student. Charcot's 1893 death left Babinski without support, and he subsequentl

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



torch.Size([1, 0])
embs.shape torch.Size([1, 0])
torch.Size([1, 0])
embs.shape torch.Size([1, 0])
torch.Size([1, 0])
embs.shape torch.Size([1, 0])

 first part 

The Shipping Forecast is a BBC Radio broadcast of weather reports and forecasts for the seas around the British Isles. It is produced by the Met Office and broadcast by BBC Radio 4 on behalf of the Maritime and Coastguard Agency. The forecast dates back over 150 years. There are currently four broadcasts per day at the following (UK local) times: 00:48, 05:20, 12:01, and 17:54. In the forecast, the waters around the British Isles are divided into 31 sea areas, also known as weather areas. The forecast begins by listing areas with gale warnings, followed by a general synopsis of pressure areas, then a forecast for each individual sea area covering wind speed and direction, precipitation, and visibility. Extended forecasts at 00:48 and 05:20 include information from coastal weather stations and inshore waters. The unique and dis